In [1]:
import pandas as pd

df = pd.read_csv("../customer_support_tickets.csv")

# Further explore to understand the relation between Ticket Ticket Subject and Description
From "Ticket Subject" and "Ticket Description" columns, how would you identify common queries from customers?
- Analyze statistics about ticket subjects
- Preprocess the ticket description
- Using TF-IDF to see the top common n-gram in ticket description of each ticket subject
- Using sentence embedding to see the top-k typical ticket descriptions for each ticket subject

In [2]:
df["Ticket Subject"].value_counts()

Ticket Subject
Refund request              576
Software bug                574
Product compatibility       567
Delivery problem            561
Hardware issue              547
Battery life                542
Network problem             539
Installation support        530
Product setup               529
Payment issue               526
Product recommendation      517
Account access              509
Peripheral compatibility    496
Data loss                   491
Cancellation request        487
Display issue               478
Name: count, dtype: int64

There are 16 different ticket subjects, evenly distributed. The content of the ticket description can be used to infer the customer's intent, which helps in categorizing the ticket subjects.

In [3]:
#Print out some example
for subject in df["Ticket Subject"].unique():
    descriptions = df[df["Ticket Subject"] == subject].head(3)["Ticket Description"]
    print(f"Some example for: {subject}")
    for d in descriptions:
        print("----------------------------------------------------------------------------------------")
        print(d)                
    print("============================================================================================")


Some example for: Product setup
----------------------------------------------------------------------------------------
I'm having an issue with the {product_purchased}. Please assist.

Your billing zip code is: 71701.

We appreciate that you have requested a website address.

Please double check your email address. I've tried troubleshooting steps mentioned in the user manual, but the issue persists.
----------------------------------------------------------------------------------------
I've recently set up my {product_purchased}, but it fails to connect to any available networks. What steps should I take to troubleshoot this issue?

I can't find the 'Product_IP' field of the I'm concerned about the security of my {product_purchased} and would like to ensure that my data is safe.
----------------------------------------------------------------------------------------
I'm having an issue with the {product_purchased}. Please assist.

Product Name: TPUBASK3E3KQ0


Join Date: Oct 2007 P

- The phrases "I'm having an issue with the {product_purchased}." and "Please assist." appear a lot, but carry little unique information.
- The ticket description contains non-text elements and potential code injections, which should be removed.

The process below lowers the text, removes special characters, lemmatizes words, and filters out non-informative phrases, yielding clean tokens for vocabulary construction and frequency counting. Stop words are retained to support a GenAI-based classifier.

In [4]:
import re
import nltk
import spacy
import torch
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from transformers import AutoTokenizer
from torchtext.vocab import build_vocab_from_iterator
from collections import Counter

In [5]:
# Custom phrases to remove (case-insensitive, regex-safe)
phrases_to_remove = [
    r"i'?m having an issue with (the )?\{?product_purchased\}?\s*\.?",
    r"please assist\.?",
    r"let me know\.?",
    r"thank you\.?",
]
phrase_pattern = re.compile(r"|".join(phrases_to_remove), re.IGNORECASE)

def clean_text(text):
    text = text.lower()
    text = phrase_pattern.sub("", text)
    text = re.sub(r"\s+", " ", text)

    return text
    
df["Ticket Content"] = df["Ticket Description"].apply(clean_text)

In [6]:
#Print out some example
for subject in df["Ticket Subject"].unique():
    descriptions = df[df["Ticket Subject"] == subject].head(3)["Ticket Content"]
    print(f"Some example for: {subject}")
    for d in descriptions:
        print("----------------------------------------------------------------------------------------")
        print(d)                
    print("============================================================================================")


Some example for: Product setup
----------------------------------------------------------------------------------------
 your billing zip code is: 71701. we appreciate that you have requested a website address. please double check your email address. i've tried troubleshooting steps mentioned in the user manual, but the issue persists.
----------------------------------------------------------------------------------------
i've recently set up my {product_purchased}, but it fails to connect to any available networks. what steps should i take to troubleshoot this issue? i can't find the 'product_ip' field of the i'm concerned about the security of my {product_purchased} and would like to ensure that my data is safe.
----------------------------------------------------------------------------------------
 product name: tpubask3e3kq0 join date: oct 2007 posts: 11,532 quote: i've recently updated the firmware of my {product_purchased}, and the issue started happening afterward. could it b

In [7]:
#Get max length of Ticket Description
df["Ticket Content"].str.split().str.len().max()

63

In [8]:
# Use TF-IDF for n-grams (2 to 4) to determine the most import n-grams for each subject
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

grouped = df.groupby("Ticket Subject")["Ticket Content"].apply(lambda texts: " ".join(texts))

vectorizer = TfidfVectorizer(ngram_range=(2, 4), stop_words="english")
X = vectorizer.fit_transform(grouped)

tfidf_df = pd.DataFrame(X.toarray(), index=grouped.index, columns=vectorizer.get_feature_names_out())

top_ngrams = {}

for subject in tfidf_df.index:
    row = tfidf_df.loc[subject]
    top = row.sort_values(ascending=False).head(10)
    top_ngrams[subject] = list(top.items())

for subject, phrases in top_ngrams.items():
    print(f"\n-Ticket Subject: {subject}")
    for phrase, score in phrases:
        print(f"{phrase}: {score:.4f}")


-Ticket Subject: Account access
ve noticed: 0.1057
issue persists: 0.0980
ve checked: 0.0842
ve tried: 0.0796
product_purchased issue: 0.0720
resolve problem: 0.0628
product_purchased account: 0.0612
works fine: 0.0612
acts unexpectedly: 0.0597
facing intermittent works: 0.0597

-Ticket Subject: Battery life
ve noticed: 0.1121
issue persists: 0.1036
product_purchased issue: 0.0923
ve tried: 0.0894
resolve problem: 0.0752
product_purchased screen: 0.0681
ve recently: 0.0653
troubleshooting steps: 0.0639
customer support: 0.0582
error message: 0.0554

-Ticket Subject: Cancellation request
issue persists: 0.1134
ve noticed: 0.0993
ve tried: 0.0914
product_purchased issue: 0.0867
ve checked: 0.0677
resolve problem: 0.0662
troubleshooting steps: 0.0583
ve recently: 0.0551
times acts unexpectedly: 0.0536
fine times: 0.0536

-Ticket Subject: Data loss
ve noticed: 0.1057
issue persists: 0.0980
resolve problem: 0.0812
ve tried: 0.0812
ve checked: 0.0812
product_purchased issue: 0.0766
product_

It seems hard to just based on top n-gram to determine the ticket subjects

In [45]:
#Use sentence embedding to determine typical tickets for each subject. 
#We first embed each sentence, get the centroid embedding by the mean of all sentence in each subject, 
#then choose top k similar sentence to the centroid

from transformers import AutoTokenizer, T5EncoderModel
from sklearn.metrics.pairwise import cosine_similarity

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/sentence-t5-base")
model = T5EncoderModel.from_pretrained("sentence-transformers/sentence-t5-base").to(device)

def get_sentence_t5_embeddings(texts, device = "cuda"):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model(**inputs)
    embeddings = output.last_hidden_state.mean(dim=1)
    return embeddings

def top_n_typical_descriptions(df, n=5, target_col = "Ticket Subject", device="cuda", batch_size=32):
    result = {}

    def get_embeddings_batched(texts):
        embeddings = []
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            with torch.no_grad():
                emb = get_sentence_t5_embeddings(batch, device=device).cpu()
            embeddings.append(emb)
        return torch.cat(embeddings, dim=0)

    for subject, group in df.groupby(target_col):
        print(f"Processing: {subject} ({len(group)} entries)")
        descriptions = group["Ticket Content"].tolist()
        
        try:
            embeddings = get_embeddings_batched(descriptions).numpy()
            centroid = embeddings.mean(axis=0, keepdims=True)
            sims = cosine_similarity(embeddings, centroid).flatten()
            top_indices = sims.argsort()[-n:][::-1]
            result[subject] = [descriptions[i] for i in top_indices]
        
        except RuntimeError as e:
            print(f"Skipping {subject} due to memory error: {e}")
            continue
    
    return result

In [46]:
result = top_n_typical_descriptions(df, target_col = "Ticket Subject")

Processing: Account access (509 entries)
Processing: Battery life (542 entries)
Processing: Cancellation request (487 entries)
Processing: Data loss (491 entries)
Processing: Delivery problem (561 entries)
Processing: Display issue (478 entries)
Processing: Hardware issue (547 entries)
Processing: Installation support (530 entries)
Processing: Network problem (539 entries)
Processing: Payment issue (526 entries)
Processing: Peripheral compatibility (496 entries)
Processing: Product compatibility (567 entries)
Processing: Product recommendation (517 entries)
Processing: Product setup (529 entries)
Processing: Refund request (576 entries)
Processing: Software bug (574 entries)


In [50]:
for key, values in result.items():
    print(f"Typical tickets for {key}")
    for value in values:
        print(f"- {value}")
    print("-" * 40)


Typical tickets for Billing inquiry
- i'm facing a problem with my {product_purchased}. the {product_purchased} is not turning on. it was working fine until yesterday, but now it doesn't respond. i am sure it works fine, but my i've already contacted customer support multiple times, but the issue remains unresolved.
-  thanks to mrnagami for the feedback. i'm having an issue on the {product_purchased}. i've tried troubleshooting steps mentioned in the user manual, but the issue persists.
- i'm facing a problem with my {product_purchased}. the {product_purchased} is not turning on. it was working fine until yesterday, but now it doesn't respond. why is it running, it asks for an the issue i'm facing is intermittent. sometimes it works fine, but other times it acts up unexpectedly.
-  sorry, the problem continues. i'm not sure why i messed up the purchase of my package sorry, the problem continues. i've followed online tutorials and community forums to troubleshoot the issue, but no luck

# Further explore to understand Ticket Type
Can you explain how you would use the "Ticket Type" field to categorize customer inquiries into broad categories or high-level categories?

- Analyze statistics related to Ticket Type.
- Analyze the relationship between Ticket Type and Ticket Subject. Is it possible to build a hierarchy?
- If not, how can the Ticket Type and Ticket Subject be used to categorized the customer tickets?


In [12]:
df["Ticket Type"].value_counts()

Ticket Type
Refund request          1752
Technical issue         1747
Cancellation request    1695
Product inquiry         1641
Billing inquiry         1634
Name: count, dtype: int64

In [38]:
import numpy as np

ticket_types = df["Ticket Type"].unique()
ticket_type_subjects = {}
for ticket_type in ticket_types:
    ticket_type_subjects[ticket_type] = set(df[df["Ticket Type"] == ticket_type]["Ticket Subject"].unique())

Jaccard_distance_matrix = np.zeros((len(ticket_types), len(ticket_types)))

def jaccard_distance(set1, set2):
    union = set1 | set2
    intersection = set1 & set2
    return 1 - len(intersection) / len(union) if union else 0

for i in range(len(ticket_types)):
    for j in range(i, len(ticket_types)):
        dist = jaccard_distance(ticket_type_subjects[ticket_types[i]],
                                ticket_type_subjects[ticket_types[j]])
        Jaccard_distance_matrix[i, j] = dist
        Jaccard_distance_matrix[j, i] = dist

df_jaccard = pd.DataFrame(Jaccard_distance_matrix, index=ticket_types, columns=ticket_types)
print(df_jaccard)

                      Technical issue  Billing inquiry  Cancellation request  \
Technical issue                   0.0              0.0                   0.0   
Billing inquiry                   0.0              0.0                   0.0   
Cancellation request              0.0              0.0                   0.0   
Product inquiry                   0.0              0.0                   0.0   
Refund request                    0.0              0.0                   0.0   

                      Product inquiry  Refund request  
Technical issue                   0.0             0.0  
Billing inquiry                   0.0             0.0  
Cancellation request              0.0             0.0  
Product inquiry                   0.0             0.0  
Refund request                    0.0             0.0  


- Each ticket subject can belong to any ticket type, so it is impossible to build a hierarchy

In [51]:
#Use sentence embedding to find top-k typical description for each ticket type
result = top_n_typical_descriptions(df, target_col = "Ticket Type")

Processing: Billing inquiry (1634 entries)
Processing: Cancellation request (1695 entries)
Processing: Product inquiry (1641 entries)
Processing: Refund request (1752 entries)
Processing: Technical issue (1747 entries)


In [85]:
for key, values in result.items():
    print(f"Typical tickets for {key}")
    for value in values:
        print(f"- {value}")
    print("-" * 40)


Typical tickets for Billing inquiry
- i'm facing a problem with my {product_purchased}. the {product_purchased} is not turning on. it was working fine until yesterday, but now it doesn't respond. i am sure it works fine, but my i've already contacted customer support multiple times, but the issue remains unresolved.
-  thanks to mrnagami for the feedback. i'm having an issue on the {product_purchased}. i've tried troubleshooting steps mentioned in the user manual, but the issue persists.
- i'm facing a problem with my {product_purchased}. the {product_purchased} is not turning on. it was working fine until yesterday, but now it doesn't respond. why is it running, it asks for an the issue i'm facing is intermittent. sometimes it works fine, but other times it acts up unexpectedly.
-  sorry, the problem continues. i'm not sure why i messed up the purchase of my package sorry, the problem continues. i've followed online tutorials and community forums to troubleshoot the issue, but no luck

- Since the content of ticket descriptions is often unclear and not specific to any subject or type, the Ticket Type field can help infer the customer's intent. 
- I would use the Ticket Type field to categorize user tickets, and treat the Ticket Subject as supplementary information to better understand the customer's situation and engage with them for further clarification if needed.
- From result of later analysis in Part II, the ticket type can be assigned to 2 high-level categories: [Technical issue, Product inquiry, Billing inquiry] and [Refund request, Cancellation request]. I propose to group [Technical issue, Product inquiry, Billing inquiry] as "Service Help" and [Refund request, Cancellation request] as [Cancellation and Refund]

# Further explore to understand Customer Satisfaction Rating
Given the "Customer Satisfaction Rating" in our dataset, how can we pre-processing this feature and how can we potentially use this feature in the models you are developing in Part II?
- The "Customer Satisfaction Rating" is an ordinal variable ranging from 1 to 5, and is only available for closed tickets. Since it is already numerical and reflects an inherent order, we can retain the original values without transformation.
- In Part II, this feature can be used to explore patterns such as which customer segments tend to give higher ratings, and to analyze how factors like resolution type or time to resolution influence customer satisfaction. It may also serve as a target variable for predictive modeling or as a signal in customer segmentation.

In [54]:
df_closed = df[df["Ticket Status"] == "Closed"]

In [91]:
df_closed["Customer Satisfaction Rating"].value_counts()

Customer Satisfaction Rating
3.0    580
1.0    553
2.0    549
5.0    544
4.0    543
Name: count, dtype: int64

In [93]:
for i in sorted(df_closed["Customer Satisfaction Rating"].unique()):
    examples = df_closed[df_closed["Customer Satisfaction Rating"] == i].head(10)
    print(f"\nSome examples for customers with rating {i}:")
    for _, e in examples.iterrows():
        # print(f"Description: {e['Ticket Description']}")
        print(f"Resolution: {e['Resolution']}")
        print("-" * 50)


Some examples for customers with rating 1.0:
Resolution: West decision evidence bit.
--------------------------------------------------
Resolution: Measure tonight surface feel forward.
--------------------------------------------------
Resolution: Measure there house management pick knowledge trade.
--------------------------------------------------
Resolution: Firm sort voice above which site arrive.
--------------------------------------------------
Resolution: Reach paper ability effort learn.
--------------------------------------------------
Resolution: Audience food pretty cut.
--------------------------------------------------
Resolution: Late center interview that.
--------------------------------------------------
Resolution: Else personal leg break book nearly.
--------------------------------------------------
Resolution: Answer story series imagine discover.
--------------------------------------------------
Resolution: Somebody east painting tree sell rock level.
-------

The resolution content appears to lack meaningful information, thus it cannot be used to predict the Customer Satisfaction Rating